In [1]:
filename = 'gs://st-work1-yuchao/author-large.txt'
author_large_no_schema = spark.read.csv(filename, header='false', sep='\t')
author_large_no_schema.createOrReplaceTempView("author_large_no_schema")
#author_large_no_schema.printSchema()
from pyspark.sql.types import *
schema = StructType([
    StructField("author", StringType(), True),    
    StructField("journal", StringType(), True),
    StructField("title", StringType(), True),
    StructField("year", LongType(), True)])

# author_large and author_large1 is same, they will be used for inner join
author_large = spark.read.csv(filename, header='false', schema=schema, sep='\t')
author_large.createOrReplaceTempView("author_large")
author_large1 = spark.read.csv(filename, header='false', schema=schema, sep='\t')
author_large1.createOrReplaceTempView("author_large1")

In [2]:
# show the all columns about author_large
spark.sql("SELECT * \
            FROM author_large \
            LIMIT 10").show()

+--------------------+--------------------+--------------------+----+
|              author|             journal|               title|year|
+--------------------+--------------------+--------------------+----+
|   Jurgen Annevelink|Modern Database S...|Object SQL - A La...|1995|
|         Rafiul Ahad|Modern Database S...|Object SQL - A La...|1995|
|      Amelia Carlson|Modern Database S...|Object SQL - A La...|1995|
|   Daniel H. Fishman|Modern Database S...|Object SQL - A La...|1995|
|  Michael L. Heytens|Modern Database S...|Object SQL - A La...|1995|
|        William Kent|Modern Database S...|Object SQL - A La...|1995|
|     Jos A. Blakeley|Modern Database S...|OQL[C++]: Extendi...|1995|
|      Yuri Breitbart|Modern Database S...|Transaction Manag...|1995|
|Hector Garcia-Molina|Modern Database S...|Transaction Manag...|1995|
|Abraham Silberschatz|Modern Database S...|Transaction Manag...|1995|
+--------------------+--------------------+--------------------+----+



Create a new table author_large2 to store the message that the same publication's two coauthors

In [3]:
spark.sql("CREATE table author_large2 (SELECT author_large.author AS author1, author_large1.author AS author2, author_large.title FROM author_large, author_large1 WHERE author_large.title = author_large1.title AND author_large.author <> author_large1.author)").show()

22/02/24 14:48:05 WARN org.apache.spark.sql.catalyst.analysis.ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.
ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/hive/conf.dist/ivysettings.xml will be used
22/02/24 14:48:08 WARN org.apache.hadoop.hive.ql.session.SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


++
||
++
++



In [4]:
spark.sql("SELECT * \
            FROM author_large2 \
            LIMIT 10").show()

+------------------+------------------+--------------------+
|           author1|           author2|               title|
+------------------+------------------+--------------------+
|Angelos Barmpoutis|    Baba C. Vemuri| +  Riemannian Me...|
|    Baba C. Vemuri|Angelos Barmpoutis| +  Riemannian Me...|
|      Yaniv Shaked|      Avishai Wool| Cipher Using OBD...|
|      Avishai Wool|      Yaniv Shaked| Cipher Using OBD...|
|  Merrick L. Furst|Jeffrey C. Jackson|          Functions.|
|  Merrick L. Furst|     Sean W. Smith|          Functions.|
|Jeffrey C. Jackson|  Merrick L. Furst|          Functions.|
|Jeffrey C. Jackson|     Sean W. Smith|          Functions.|
|     Sean W. Smith|  Merrick L. Furst|          Functions.|
|     Sean W. Smith|Jeffrey C. Jackson|          Functions.|
+------------------+------------------+--------------------+



In [35]:
spark.sql("CREATE table top_coauthor (SELECT author1, author2, count(title) AS counts FROM author_large2 GROUP BY author1, author2 ORDER BY counts DESC LIMIT 20)").show()

22/02/23 16:32:41 WARN org.apache.spark.sql.catalyst.analysis.ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.


++
||
++
++



In [61]:
spark.sql("SELECT * FROM top_coauthor").show()

+--------------------+--------------------+------+
|             author1|             author2|counts|
+--------------------+--------------------+------+
|      Irith Pomeranz|   Sudhakar M. Reddy|   249|
|   Sudhakar M. Reddy|      Irith Pomeranz|   249|
|   Divyakant Agrawal|       Amr El Abbadi|   161|
|       Amr El Abbadi|   Divyakant Agrawal|   161|
|     Makoto Takizawa|      Tomoya Enokido|   141|
|      Tomoya Enokido|     Makoto Takizawa|   141|
|       Didier Dubois|         Henri Prade|   122|
|         Henri Prade|       Didier Dubois|   122|
|    Tharam S. Dillon|     Elizabeth Chang|   118|
|     Elizabeth Chang|    Tharam S. Dillon|   118|
|       Kee-Young Yoo|       Hyun-Sung Kim|   111|
|       Hyun-Sung Kim|       Kee-Young Yoo|   111|
|     Mary Jane Irwin|Narayanan Vijaykr...|   107|
|Narayanan Vijaykr...|     Mary Jane Irwin|   107|
|     Mary Jane Irwin|  Mahmut T. Kandemir|   100|
|  Mahmut T. Kandemir|     Mary Jane Irwin|   100|
|  Maurizio Lenzerini| Giuseppe

In [60]:
# remove the repeat rows so we can get the top10 co-authors table
spark.sql("SELECT * FROM top_coauthor WHERE author1 in ('Sudhakar M. Reddy','Amr El Abbadi','Tomoya Enokido','Henri Prade','Elizabeth Chang','Hyun-Sung Kim','Mary Jane Irwin','Maurizio Lenzerini','Chun Chen')").show()

+------------------+--------------------+------+
|           author1|             author2|counts|
+------------------+--------------------+------+
| Sudhakar M. Reddy|      Irith Pomeranz|   249|
|     Amr El Abbadi|   Divyakant Agrawal|   161|
|    Tomoya Enokido|     Makoto Takizawa|   141|
|       Henri Prade|       Didier Dubois|   122|
|   Elizabeth Chang|    Tharam S. Dillon|   118|
|     Hyun-Sung Kim|       Kee-Young Yoo|   111|
|   Mary Jane Irwin|Narayanan Vijaykr...|   107|
|   Mary Jane Irwin|  Mahmut T. Kandemir|   100|
|Maurizio Lenzerini| Giuseppe De Giacomo|    99|
|         Chun Chen|           Jiajun Bu|    99|
+------------------+--------------------+------+

